# ParOSSE Demonstration: Simulated IR / MW Retrieval

This simple experiment exercises each component of the ParOSSE workflow for a simplified IR sensor similar to the CrIS instrument.

The procedure, common to each architecture evaluation experiment, is:
1. Define the data, models, retrieval, and analysis to be used in the experiment
2. Read from nature run
3. Run forward model(s)
4. Run instrument model(s)
5. Run retrieval(s)
6. Compute diagnostics
7. Produce plots and statistics

This particular run bypasses the detailed RT model, instrument model, and retrieval and instead emulates what the retrieved field would look like by applying smoothing and noise.

Experiment settings are contained in the file "parosse.yml"

Derek Posselt
JPL
September 2024

In [14]:
# Import system libraries
from time import time
import logging
import os
import yaml
import pathlib
import numpy as np
import pandas as pd

# Nature run subsystem
from nature_runs.create_nr_config import create_nr_config
from nature_runs.pre_process_nr import pre_process_nr

# Forward / radiative transfer model subsystem
from forward_models.fwd_temp_rh import fwd_temp_rh
from forward_models.post_process_rtm import post_process_rtm

# Instrument model subsystem
from instrument_models.create_instrument_config import create_instrument_config
from instrument_models.gauss_filt_temp_rh import gauss_filt_temp_rh

# Retrieval subsystem
from retrievals.simple_sounder_temp_rh_retr import simple_sounder_temp_rh_retr

# Analysis and plotting subsystem
from analysis_functions.rmse_and_yield import rmse_and_yield
from analysis_functions.rmse_and_yield_latlon import rmse_and_yield_latlon
from analysis_functions.plot_temp_rh import plot_temp_rh

# Utilities
from utils.logger_config import configure_logger, crashed_log
from utils.data_handling import copy_by_keys, csv_get_nth_row, csv_get_number_rows

# Parmap - used here for vertical interpolation
from Parmap.parmap import parmap

# Plotting routines
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# Matplotlib settings
# Tell matplotlib to use truetype fonts (better for Illustrator)
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

# Set the font to something Illustrator will understand...
mpl.rcParams['font.sans-serif'] = "Arial"
# Force to always use sans-serif fonts
mpl.rcParams['font.family'] = "sans-serif"

# Set the axis tick labels larger
mpl.rc('xtick', labelsize=20)
mpl.rc('ytick', labelsize=20)


In [2]:
def log_config_keys(config_dict, config_name):
    """
    Logs the keys and values of the given configuration dictionary.

    :param config_dict: The configuration dictionary to log.
    :param config_name: A label for the type of config (e.g., 'experiment', 'Nature Run').
    """
    logger.info(f'Keys in {config_name} configuration:')
    for key, value in config_dict.items():
        logger.info(f'  {key}: {value}')



In [3]:
def experiment_config(experiment_config_file, csv_line=None):
    """
    Populates configuration dictionaries for the experiment.

    The configurations consist of:
    1. experiment_config: General settings for the entire workflow.
    2. nr_config: Settings related to the Nature Run (NR) subsystem.
    3. Instrument model settings (added to the experiment config).
    4. Retrieval settings (currently none).
    5. Plotting and analysis settings (added to the experiment config).
    """

    # -------------------------------------------------------------------
    # Initialize the NR config dictionary
    # -------------------------------------------------------------------
    nr_config = None

    # -------------------------------------------------------------------
    # Load general experiment settings from the config file (CSV or YAML)
    # -------------------------------------------------------------------
    if experiment_config_file.endswith('.csv'):
        assert csv_line is not None, "csv_line must be provided for CSV input."
        logger.info(f"Loading configuration from CSV: {experiment_config_file}, line {csv_line}")
        nr_config, experiment_config = csv_get_nth_row(experiment_config_file, csv_line)
    else:
        logger.info(f"Loading configuration from YAML: {experiment_config_file}")
        with open(experiment_config_file, 'r') as config_file:
            experiment_config = yaml.safe_load(config_file)

    # Log the contents of the experiment configuration
    log_config_keys(experiment_config, "experiment")

    # -------------------------------------------------------------------
    # Define settings for the Nature Run Subsystem (NR)
    # -------------------------------------------------------------------
    # Update experiment and NR config with Nature Run settings
    nr_config, experiment_config = create_nr_config(experiment_config, nr_config=nr_config)

    # Make sure that the output/plot directory exists.
    output_path = pathlib.Path(experiment_config['output_path'])
    output_path.mkdir(parents=True, exist_ok=True)

    plot_path = pathlib.Path(experiment_config['plot_path'])
    plot_path.mkdir(parents=True, exist_ok=True)

    # Log the contents of the NR configuration
    log_config_keys(nr_config, "Nature Run")

    # -------------------------------------------------------------------
    # Define settings for the Instrument Model Subsystem
    # -------------------------------------------------------------------
    # Add satellite instrument information to the experiment config
    experiment_config = create_instrument_config(experiment_config)

    # Log the contents of the updated experiment configuration
    log_config_keys(experiment_config, "experiment")

    return experiment_config, nr_config



In [4]:
# -----------------------------------
# Function that Runs the Forward Model Subsystem
# -----------------------------------
def run_fwd_model(nr_fwd_run):

  """
  This will run the Forward Model Subsystem
  For a single nx_tile x ny_tile section of the nature run
  1. Extract data from the input list
  2. Run the forward model
  3. Insert data into the output dictionary
  """
  # Extract data from the input list
  expt_config = nr_fwd_run[0]
  nr_config   = nr_fwd_run[1]
  nr_data     = nr_fwd_run[2]
  run_num     = nr_fwd_run[3]

  # Run the forward model.
  # In this case, this consists just of computing RH and interpolating in height
  rtm_data = fwd_temp_rh(nr_data, nr_config, expt_config, dz=nr_config['dz'], ztop=nr_config['ztop'])

  # Put the tile indices and dimensions into the rtm_data dictionary
  copy_by_keys(nr_data, rtm_data, ['ix', 'jy', 'nx_tile', 'ny_tile'])

  # Return the rtm data dictionary
  return rtm_data


In [5]:
def get_output_columns(retr_data):
    rmse_output = OUTPUT_COLUMNS
    return {key: retr_data[key] for key in rmse_output if key in retr_data}


In [ ]:
"""
The workflow consists of 7 components:
1. Fill the configuration dictionaries
2. Pre-process the nature run data
3. Run the forward model
4. Post-process the forward model output
5. Run the instrument model
6. Run the retrieval
7. Run the analysis and plotting routine(s)
"""

# Set logging
logger = logging.getLogger(__name__)

exclude_keywords = ['readers', 'matplotlib']

OUTPUT_FILE_COLUMNS = ['nr_expt','nr_date','nr_time','sat_type','i1','i2','j1','j2',]
OUTPUT_COLUMNS = ['rmse_temp', 'rmse_h2o', 'rmse_rh', 'yield_temp', 'yield_h2o', 'yield_rh']

LOG_FILE = None

# Set the root path
parosse_path = '/Users/derekp/parosse-noaa/'

# Set the root name of the yaml config file
yaml_file = parosse_path+'parosse.yml'

# Define the logging level
log_level=1

configure_logger(exclude_keywords, log_level)

# If the input file is not found, exit.
if not os.path.exists(yaml_file):
    raise ValueError(f"Error: File {yaml_file} does not exist.")

logger.info(f"Processing YAML file: {yaml_file}")


In [ ]:
# Start the timer
t0 = time()

# Configure the experiment
expt_config, nr_config = experiment_config(yaml_file)


In [ ]:
# -------------------------------------------------------------------
# Run the Nature Run Sub-System
# -------------------------------------------------------------------

# Pre-process the nature run data for ingest into parmap
# This needs doing, even if data is being read from file
nr_fwd_runs, nr_dims = pre_process_nr(nr_config,expt_config)


In [ ]:
# -------------------------------------------------------------------
# Run the Radiative Transfer Model Sub-System
# -------------------------------------------------------------------

# If user has not requested RTM data be read from file, run RTM
if ( not expt_config['read_RTM']):
  # Initialize parmap
  pmap = parmap.Parmap(mode=expt_config['parmode'], numWorkers=expt_config['num_workers'])

  # Increment the timer
  t1 = time()

  # Run the RTM, returning a list of output dictionaries
  rtm_data_list = pmap(run_fwd_model, nr_fwd_runs)

  # Increment the timer
  t2 = time()

# User has requested RTM data be read
else:
  # Create an empty data list and increment the timers
  rtm_data_list = []
  t1 = time()
  t2 = time()

# If user has requested RTM be run, then reconstruct the RTM data and write it to file
# Otherwise, read data from file
# Both happen within the post-processing routine
rtm_data = post_process_rtm(rtm_data_list, expt_config, nr_config, nr_dims)

# Increment the timer
t3 = time()

In [ ]:
# -------------------------------------------------------------------
# Run the Instrument Model Subsystem - No Parallelism
# -------------------------------------------------------------------
# The instrument model simply applies gaussian filtering to the "RTM" output
inst_data = gauss_filt_temp_rh(rtm_data, nr_config, expt_config)

# Increment the timer
t4 = time()

In [ ]:
# -------------------------------------------------------------------
# Run the Retrieval Subsystem
# -------------------------------------------------------------------
retr_data = simple_sounder_temp_rh_retr(expt_config, nr_config, rtm_data, inst_data)

# Increment the timer
t5 = time()


In [ ]:
# -------------------------------------------------------------------
# Run the Plotting and Analysis Subsystem
# -------------------------------------------------------------------
# Compute RMSE
if nr_config['grid_type'] == 'latlon':
  retr_data = rmse_and_yield_latlon(expt_config, rtm_data, retr_data)
else:
  retr_data = rmse_and_yield(expt_config, rtm_data, retr_data)

output_status = plot_temp_rh(nr_config, expt_config, rtm_data, retr_data, inst_data=inst_data)

# Increment the timer
t6 = time()


In [ ]:
# Extract data from dictionaries
model_prefix = f"{nr_config['file_prefix']}_{nr_config['i1']}i1_{nr_config['i2']}i2_{nr_config['j1']}j1_{nr_config['j2']}j2"
sat_type = expt_config['sat_type']
x_res = expt_config['x_res']
z_res = expt_config['z_res']
xvec = expt_config['xvec'] # This will be a 2D array in the case of lat/lon grids
yvec = expt_config['yvec']
zvec = expt_config['zvec']

logger.info(f"Length of xvec: {len(xvec)} yvec: {len(yvec)} zvec: {len(zvec)}")

y_idx = np.int32(np.floor(nr_config['y_idx'] / nr_config['icoarse']))
z_idx = nr_config['z_idx']
if pd.isnull(nr_config['ztop']):
  ztop = 20000
else:
  ztop = nr_config['ztop']

if nr_config['y_idx'] < 0:
  y_idx = np.int32(np.floor(rtm_data['ny']/2/nr_config['icoarse']))

# Set additional y indices
y_incices = [
  y_idx,
  np.int32(np.floor(rtm_data['ny']/4)), # 1/4 of the way through the domain
  np.int32(np.floor(rtm_data['ny']/4))+np.int32(np.floor(rtm_data['ny']/2)) # 3/4 of the way through the domain
]

logger.info(f'Index of y-cross-section: {y_idx}')

iout = expt_config['iout'] # This will be a vector in the case of lat/lon grid
jout = expt_config['jout']
kout = expt_config['kout']
plot_path = expt_config['plot_path']
h2o_var = expt_config['h2o_var']
rh_var = expt_config['rh_var']
temp_var = expt_config['temp_var']

# Obtain lat and lon arrays
xlat = retr_data[retr_data['lat_var']]
xlon = retr_data[retr_data['lon_var']]



In [ ]:
#----------------------------------------------
# PRODUCE RMSE AND YIELD PLOTS
#----------------------------------------------

# Plot the RMSE profiles for temperature and water vapor
rmse_temp_prof = retr_data['rmse_temp_prof']
rmse_temp = retr_data['rmse_temp']
rmse_h2o_prof  = retr_data['rmse_h2o_prof']
rmse_h2o  = retr_data['rmse_h2o']
rmse_rh_prof  = retr_data['rmse_rh_prof']
rmse_rh  = retr_data['rmse_rh']
yield_temp_prof = retr_data['yield_temp_prof'] * 100.0 # Convert to percent
yield_temp = retr_data['yield_temp'] * 100.0 # Convert to percent

# Plot temperature RMSE profile
rmse_fig = plt.figure(figsize=(3,6))
plt.plot(rmse_temp_prof,zvec[::kout])
plt.xlim(0,7.0)
plt.ylim(0,ztop)
plt.ylabel('Height (m)', fontsize=24)
plt.xlabel('T RMSE (K)', fontsize=24)
plt.title(f'T RMSE: {rmse_temp:.2f}', fontsize=24)

# Plot RH RMSE profile
rmse_fig = plt.figure(figsize=(3,6))
plt.plot(rmse_rh_prof,zvec[::kout])
plt.xlim(0,15.0)
plt.ylim(0,ztop)
plt.ylabel('Height (m)', fontsize=24)
plt.xlabel('RH RMSE (%)', fontsize=24)
plt.title(f'RH RMSE: {rmse_rh:.2f}', fontsize=24)

# Plot vapor RMSE profile
rmse_fig = plt.figure(figsize=(3,6))
plt.plot(rmse_h2o_prof,zvec[::kout])
plt.xlim(0,5.0)
plt.ylim(0,ztop)
plt.ylabel('Height (m)', fontsize=24)
plt.xlabel('Qv RMSE (%)', fontsize=24)
plt.title(f'Qv RMSE: {rmse_h2o:.2f}', fontsize=24)

# Plot the yield profile - should be the same for both temperature and water vapor
yield_fig = plt.figure(figsize=(3,6))
plt.plot(yield_temp_prof,zvec[::kout])
plt.xlim(0,105.0)
plt.ylim(0,ztop)
plt.ylabel('Height (m)', fontsize=24)
plt.xlabel('Yield (%)', fontsize=24)
plt.title(f'Retrieval Yield: {yield_temp:.2f}', fontsize=24)



In [ ]:
#----------------------------------------------
# PRODUCE PLAN VIEW PLOTS
#----------------------------------------------

# Plot the nature run surface temperature
Tplot = np.squeeze(rtm_data[temp_var][z_idx,:,:])
Tmin = np.floor(np.min(Tplot))
Tmax = np.ceil(np.max(Tplot))

logger.info(f'Min,max T: {Tmin} {Tmax}')

# Plot the temperature distribution
tfig = plt.figure(figsize=(20,15))
if nr_config['grid_type'] == 'latlon':
  plt.pcolormesh(xlon[0,:],xlat[:,0],Tplot,vmin=Tmin,vmax=Tmax, cmap='RdYlBu_r')
  plt.ylabel('Latitude  (deg)', fontsize=36)
  plt.xlabel('Longitude (deg)', fontsize=36)
else:
  plt.pcolormesh(xvec,yvec,Tplot,vmin=Tmin,vmax=Tmax, cmap='RdYlBu_r')
  plt.ylabel('Distance (km)', fontsize=36)
  plt.xlabel('Distance (km)', fontsize=36)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
plt.title('Surface Temperature', fontsize=40)
plt.colorbar().set_label(label='Temperature (K)',size=36)

# Plot the filtered non-noisy surface temperature distribution, if available
if inst_data is not None:
  Tplot = np.squeeze(inst_data[temp_var][z_idx,:,:])
  Tmin = np.floor(np.min(Tplot))
  Tmax = np.ceil(np.max(Tplot))

  # print('Z index: ',z_idx)
  logger.info(f'Min,max T: {Tmin} {Tmax}')
  tfig = plt.figure(figsize=(20,15))
  if nr_config['grid_type'] == 'latlon':
    jy = np.int32(np.floor(0.5 * len(Tplot[:,0])))
    logger.info(f'Dimensions of lat and lon, jy, jout, iout: {np.shape(xlat)} {np.shape(xlon)} {jy} {jout} {iout}')
    xlat1d = np.squeeze(xlat[0::jout,0])
    xlon1d = np.squeeze(xlon[jy,0::iout[jy]])
    logger.info(f'Dimensions of reduced size lat and lon {len(xlat1d)} {len(xlon1d)}')
    plt.pcolormesh(xlon1d,xlat1d,Tplot[0::jout,0::iout[jy]],vmin=Tmin,vmax=Tmax, cmap='RdYlBu_r')
    plt.ylabel('Latitude  (deg)', fontsize=36)
    plt.xlabel('Longitude (deg)', fontsize=36)
  else:
    plt.pcolormesh(xvec[0::iout],yvec[0::jout],Tplot[0::jout,0::iout],vmin=Tmin,vmax=Tmax, cmap='RdYlBu_r')
    plt.ylabel('Distance (km)', fontsize=36)
    plt.xlabel('Distance (km)', fontsize=36)
  plt.yticks(fontsize=32)
  plt.xticks(fontsize=32)
  plt.title('Surface Temperature, Filtered', fontsize=40)
  plt.colorbar().set_label(label='Temperature (K)',size=36)

# Plot the noisy surface temperature
Tplot = np.squeeze(retr_data[temp_var][z_idx,:,:])

# Plot the filtered noisy temperature distribution
tfig = plt.figure(figsize=(20,15))
if nr_config['grid_type'] == 'latlon':
  jy = np.int32(np.floor(0.5 * len(Tplot[:,0])))
  xlat1d = np.squeeze(xlat[0::jout,0])
  xlon1d = np.squeeze(xlon[jy,0::iout[jy]])
  plt.pcolormesh(xlon1d,xlat1d,Tplot[0::jout,0::iout[jy]],vmin=Tmin,vmax=Tmax, cmap='RdYlBu_r')
  plt.ylabel('Latitude  (deg)', fontsize=36)
  plt.xlabel('Longitude (deg)', fontsize=36)
else:
  plt.pcolormesh(xvec[0::iout],yvec[0::jout],Tplot[0::jout,0::iout],vmin=Tmin,vmax=Tmax, cmap='RdYlBu_r')
  plt.ylabel('Distance (km)', fontsize=36)
  plt.xlabel('Distance (km)', fontsize=36)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
plt.title('Surface Temperature, Retrieved', fontsize=40)
plt.colorbar().set_label(label='Temperature (K)',size=36)

# Plot the unfiltered cloud top height on a range from 0 to 20 km
CTmin = 0.0
CTmax = 20000.0
ctfig = plt.figure(figsize=(20,15))
if nr_config['grid_type'] == 'latlon':
  plt.pcolormesh(xlon[0,:],xlat[:,0],rtm_data['cth'],vmin=CTmin,vmax=CTmax, cmap='RdYlBu_r')
  plt.ylabel('Latitude  (deg)', fontsize=36)
  plt.xlabel('Longitude (deg)', fontsize=36)
else:
  plt.pcolormesh(xvec,yvec,rtm_data['cth'],vmin=CTmin,vmax=CTmax, cmap='RdYlBu_r')
  plt.ylabel('Distance (km)', fontsize=36)
  plt.xlabel('Distance (km)', fontsize=36)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
plt.title('Cloud Top Height', fontsize=40)
plt.colorbar().set_label(label='Height (m)',size=36)

# Plot the filtered cloud top height on a range from 0 to 20 km
CTmin = 0.0
CTmax = 20000.0
ctfig = plt.figure(figsize=(20,15))
if nr_config['grid_type'] == 'latlon':
  jy = np.int32(np.floor(0.5 * len(Tplot[:,0])))
  xlat1d = np.squeeze(xlat[0::jout,0])
  xlon1d = np.squeeze(xlon[jy,0::iout[jy]])
  plt.pcolormesh(xlon1d,xlat1d,retr_data['cth'][0::jout,0::iout[jy]],vmin=CTmin,vmax=CTmax, cmap='RdYlBu_r')
  plt.ylabel('Latitude  (deg)', fontsize=36)
  plt.xlabel('Longitude (deg)', fontsize=36)
else:
  plt.pcolormesh(xvec[0::iout],yvec[0::iout],retr_data['cth'][0::jout,0::iout],vmin=CTmin,vmax=CTmax, cmap='RdYlBu_r')
  plt.ylabel('Distance (km)', fontsize=36)
  plt.xlabel('Distance (km)', fontsize=36)
plt.yticks(fontsize=32)
plt.xticks(fontsize=32)
plt.title('Cloud Top Height, Filtered', fontsize=40)
plt.colorbar().set_label(label='Height (m)',size=36)



In [ ]:
#----------------------------------------------
# PRODUCE CROSS-SECTION PLOTS
#----------------------------------------------
# Loop over all y indices
for y_idx in y_incices:

  # Plot the nature run water vapor cross-section
  qfig = plt.figure(figsize=(20,6))
  Qvplot = rtm_data[h2o_var][:,y_idx,:]
  Qvmin = np.floor(np.min(Qvplot))
  Qvmax = np.ceil(np.max(Qvplot))
  if nr_config['grid_type'] == 'latlon':
    plt.pcolormesh(xvec[y_idx],zvec,Qvplot,vmin=Qvmin,vmax=Qvmax, cmap='viridis')
  else:
    plt.pcolormesh(xvec,zvec,Qvplot,vmin=Qvmin,vmax=Qvmax, cmap='viridis')
  plt.ylim(0,ztop)
  plt.ylabel('Height (m)', fontsize=24)
  plt.xlabel('km', fontsize=24)
  plt.title('Water Vapor Slice, y='+str(y_idx), fontsize=24)
  plt.colorbar().set_label(label='%',size=20)

  # Plot the filtered non-noisy water vapor cross-section, if available - use NR min/max
  if inst_data is not None:
    qfig = plt.figure(figsize=(20,6))
    if nr_config['grid_type'] == 'latlon':
      Qvplot = inst_data[h2o_var][0::kout,y_idx,0::iout[y_idx]]
      plt.pcolormesh(xvec[y_idx][0::iout[y_idx]],zvec[0::kout],Qvplot,vmin=Qvmin,vmax=Qvmax, cmap='viridis')
    else:
      Qvplot = inst_data[h2o_var][0::kout,y_idx,0::iout]
      plt.pcolormesh(xvec[0::iout],zvec[0::kout],Qvplot,vmin=Qvmin,vmax=Qvmax, cmap='viridis')
    plt.ylim(0,ztop)
    plt.ylabel('Height (m)', fontsize=24)
    plt.xlabel('km', fontsize=24)
    plt.title('Water Vapor Slice, Filtered, y='+str(y_idx), fontsize=24)
    plt.colorbar().set_label(label='%',size=20)

  # Plot the noisy "retrieved" water vapor - use NR min/max
  qfig = plt.figure(figsize=(20,6))
  if nr_config['grid_type'] == 'latlon':
    Qvplot = retr_data[h2o_var][0::kout,y_idx,0::iout[y_idx]]
    plt.pcolormesh(xvec[y_idx][0::iout[y_idx]],zvec[0::kout],Qvplot,vmin=Qvmin,vmax=Qvmax, cmap='viridis')
  else:
    Qvplot = retr_data[h2o_var][0::kout,y_idx,0::iout]
    plt.pcolormesh(xvec[0::iout],zvec[0::kout],Qvplot,vmin=Qvmin,vmax=Qvmax, cmap='viridis')
  plt.ylim(0,ztop)
  plt.ylabel('Height (m)', fontsize=24)
  plt.xlabel('km', fontsize=24)
  plt.title('Water Vapor Retrieved, y='+str(y_idx), fontsize=24)
  plt.colorbar().set_label(label='%',size=20)

  # Plot the nature run RH cross-section
  qfig = plt.figure(figsize=(20,6))
  if nr_config['grid_type'] == 'latlon':
    plt.pcolormesh(xvec[y_idx],zvec,rtm_data[rh_var][:,y_idx,:],vmin=0.0,vmax=100.0, cmap='viridis')
  else:
    plt.pcolormesh(xvec,zvec,rtm_data[rh_var][:,y_idx,:],vmin=0.0,vmax=100.0, cmap='viridis')
  plt.ylim(0,ztop)
  plt.ylabel('Height (m)', fontsize=24)
  plt.xlabel('km', fontsize=24)
  plt.title('Relative Humidity Slice, y='+str(y_idx), fontsize=24)
  plt.colorbar().set_label(label='%',size=20)

  # Plot the filtered non-noisy RH cross-section, if available
  if inst_data is not None:
    qfig = plt.figure(figsize=(20,6))
    if nr_config['grid_type'] == 'latlon':
      plt.pcolormesh(xvec[y_idx][0::iout[y_idx]],zvec[0::kout],inst_data[rh_var][0::kout,y_idx,0::iout[y_idx]],vmin=0.0,vmax=100.0, cmap='viridis')
    else:
      plt.pcolormesh(xvec[0::iout],zvec[0::kout],inst_data[rh_var][0::kout,y_idx,0::iout],vmin=0.0,vmax=100.0, cmap='viridis')
    plt.ylim(0,ztop)
    plt.ylabel('Height (m)', fontsize=24)
    plt.xlabel('km', fontsize=24)
    plt.title('Relative Humidity Slice, Filtered, y='+str(y_idx), fontsize=24)
    plt.colorbar().set_label(label='%',size=20)

  # Plot the noisy "retrieved" RH
  qfig = plt.figure(figsize=(20,6))
  if nr_config['grid_type'] == 'latlon':
    plt.pcolormesh(xvec[y_idx][0::iout[y_idx]],zvec[0::kout],retr_data[rh_var][0::kout,y_idx,0::iout[y_idx]],vmin=0.0,vmax=100.0, cmap='viridis')
  else:
    plt.pcolormesh(xvec[0::iout],zvec[0::kout],retr_data[rh_var][0::kout,y_idx,0::iout],vmin=0.0,vmax=100.0, cmap='viridis')
  plt.ylim(0,ztop)
  plt.ylabel('Height (m)', fontsize=24)
  plt.xlabel('km', fontsize=24)
  plt.title('Relative Humidity Retrieved, y='+str(y_idx), fontsize=24)
  plt.colorbar().set_label(label='%',size=20)




In [ ]:
# Print timing
logger.info(f'Time to initialize:           {t1-t0}')
logger.info(f'Time to run RTM:              {t2-t1}')
logger.info(f'Time to reconstruct RTM:      {t3-t2}')
logger.info(f'Time to run instrument model: {t4-t3}')
logger.info(f'Time to run retrieval:        {t5-t4}')
logger.info(f'Time to run analysis:         {t6-t5}')
logger.info(f'Total run time:               {t6-t0}')
